In [175]:
import pandas as pd
import numpy as np
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from math import sqrt

In [176]:
dataset=pd.read_csv("Dataset/DataCoSupplyChainDataset.csv",header= 0,encoding= 'unicode_escape')
dataset.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [177]:
dataset.shape

(180519, 53)

In [178]:
dataset['Customer Full Name'] = dataset['Customer Fname'].astype(str)+dataset['Customer Lname'].astype(str)
data=dataset.drop(columns=['Customer Fname','Customer Lname','Order Zipcode', 'Product Description','Customer Email','Customer Password','Customer Street','Latitude','Longitude','Order Id','Order Item Cardprod Id','Order Zipcode','Product Description','Product Image'])

In [179]:
data.shape

(180519, 42)

In [180]:
data.isnull().sum()

Type                             0
Days for shipping (real)         0
Days for shipment (scheduled)    0
Benefit per order                0
Sales per customer               0
Delivery Status                  0
Late_delivery_risk               0
Category Id                      0
Category Name                    0
Customer City                    0
Customer Country                 0
Customer Id                      0
Customer Segment                 0
Customer State                   0
Customer Zipcode                 3
Department Id                    0
Department Name                  0
Market                           0
Order City                       0
Order Country                    0
Order Customer Id                0
order date (DateOrders)          0
Order Item Discount              0
Order Item Discount Rate         0
Order Item Id                    0
Order Item Product Price         0
Order Item Profit Ratio          0
Order Item Quantity              0
Sales               

In [181]:
data = data.dropna()
data.isnull().sum()
data.shape

(180516, 42)

In [182]:
data=data.rename(columns = {'order date (DateOrders)': 'Order_Date'}, inplace = False)

In [183]:
def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
               # deleting the column from the dataset
    return(list(col_corr))
cols=correlation(data,0.5)

In [184]:
print(cols)

['Order Customer Id', 'Order Profit Per Order', 'Order Item Discount Rate', 'Order Item Total', 'Product Price', 'Sales', 'Product Card Id', 'Days for shipment (scheduled)', 'Department Id', 'Order Item Product Price', 'Order Item Profit Ratio', 'Product Category Id']


In [185]:
#new=data.loc[data['Department Id'] == 2]
final_cols=['Order_Date','Order Item Quantity','Department Id','Product Category Id', 'Order Customer Id',  'Days for shipment (scheduled)', 'Order Item Profit Ratio', 'Product Card Id', 'Sales', 'Order Item Discount Rate', 'Order Item Product Price']


In [186]:
data=data[final_cols]

In [187]:
data.head()

,Order_Date,Order Item Quantity,Department Id,Product Category Id,Order Customer Id,Days for shipment (scheduled),Order Item Profit Ratio,Product Card Id,Sales,Order Item Discount Rate,Order Item Product Price
0,1/31/2018 22:56,1,2,73,20755,4,0.29,1360,327.75,0.04,327.75
1,1/13/2018 12:27,1,2,73,19492,4,-0.80,1360,327.75,0.05,327.75
2,1/13/2018 12:06,1,2,73,19491,4,-0.80,1360,327.75,0.06,327.75
3,1/13/2018 11:45,1,2,73,19490,4,0.08,1360,327.75,0.07,327.75
4,1/13/2018 11:24,1,2,73,19489,4,0.45,1360,327.75,0.09,327.75


In [188]:
numerical_features = [f for f in data.columns if data[f].dtypes!='O']
only=data[numerical_features]
only.head()

,Order Item Quantity,Department Id,Product Category Id,Order Customer Id,Days for shipment (scheduled),Order Item Profit Ratio,Product Card Id,Sales,Order Item Discount Rate,Order Item Product Price
0,1,2,73,20755,4,0.29,1360,327.75,0.04,327.75
1,1,2,73,19492,4,-0.80,1360,327.75,0.05,327.75
2,1,2,73,19491,4,-0.80,1360,327.75,0.06,327.75
3,1,2,73,19490,4,0.08,1360,327.75,0.07,327.75
4,1,2,73,19489,4,0.45,1360,327.75,0.09,327.75


In [189]:
only=only.drop(columns=['Department Id','Order Item Quantity','Product Category Id','Order Customer Id','Product Card Id'])
only_features = [f for f in only.columns ]
values = only.values
values = values.reshape((len(values), len(only_features)))
only.head()

,Days for shipment (scheduled),Order Item Profit Ratio,Sales,Order Item Discount Rate,Order Item Product Price
0,4,0.29,327.75,0.04,327.75
1,4,-0.80,327.75,0.05,327.75
2,4,-0.80,327.75,0.06,327.75
3,4,0.08,327.75,0.07,327.75
4,4,0.45,327.75,0.09,327.75


In [190]:
# # train the standardization
scaler = StandardScaler()
scaler = scaler.fit(values)

# # standardization the dataset and print the first 5 rows
normalized = scaler.transform(values)
for i in range(5):
	print(normalized[i])

[ 0.77715354  0.36280374  0.93729177 -0.87578036  1.33483273]
[ 0.77715354 -1.97225495  0.93729177 -0.73376381  1.33483273]
[ 0.77715354 -1.97225495  0.93729177 -0.59174732  1.33483273]
[ 0.77715354 -0.08706994  0.93729177 -0.44973079  1.33483273]
[ 0.77715354  0.70556464  0.93729177 -0.16569769  1.33483273]


In [191]:
# # inverse transform and print the first 5 rows
# inversed = scaler.inverse_transform(normalized)
# for i in range(5):
# 	print(inversed[i])

In [192]:
x=pd.DataFrame(data=normalized,index=range(0,len(only)),columns=only_features)
x.head()
# #print(only['i'])

# for i in x.columns:
#     data=data.assign(i=x[i])
#     #print(i)
# data.head()


# data.head() 

# print(len(data.columns))
# data.isnull().sum()
# data = data.dropna()

,Days for shipment (scheduled),Order Item Profit Ratio,Sales,Order Item Discount Rate,Order Item Product Price
0,0.777154,0.362804,0.937292,-0.875780,1.334833
1,0.777154,-1.972255,0.937292,-0.733764,1.334833
2,0.777154,-1.972255,0.937292,-0.591747,1.334833
3,0.777154,-0.087070,0.937292,-0.449731,1.334833
4,0.777154,0.705565,0.937292,-0.165698,1.334833


In [193]:
data.drop(labels=x.columns, axis="columns", inplace=True)
data[x.columns] = x[x.columns]
# for i in x.columns:
#     data.assign(i=x[i])
data.head()

,Order_Date,Order Item Quantity,Department Id,Product Category Id,Order Customer Id,Product Card Id,Days for shipment (scheduled),Order Item Profit Ratio,Sales,Order Item Discount Rate,Order Item Product Price
0,1/31/2018 22:56,1,2,73,20755,1360,0.777154,0.362804,0.937292,-0.875780,1.334833
1,1/13/2018 12:27,1,2,73,19492,1360,0.777154,-1.972255,0.937292,-0.733764,1.334833
2,1/13/2018 12:06,1,2,73,19491,1360,0.777154,-1.972255,0.937292,-0.591747,1.334833
3,1/13/2018 11:45,1,2,73,19490,1360,0.777154,-0.087070,0.937292,-0.449731,1.334833
4,1/13/2018 11:24,1,2,73,19489,1360,0.777154,0.705565,0.937292,-0.165698,1.334833


In [194]:
#data=data.set_index('Order_Date')
data.Order_Date = pd.DatetimeIndex(data.Order_Date).to_period('D')
#data.sort_index(inplace=True)
#data=data.sort_values(by = 'Order_Date') 

In [195]:
data.head()

,Order_Date,Order Item Quantity,Department Id,Product Category Id,Order Customer Id,Product Card Id,Days for shipment (scheduled),Order Item Profit Ratio,Sales,Order Item Discount Rate,Order Item Product Price
0,2018-01-31,1,2,73,20755,1360,0.777154,0.362804,0.937292,-0.875780,1.334833
1,2018-01-13,1,2,73,19492,1360,0.777154,-1.972255,0.937292,-0.733764,1.334833
2,2018-01-13,1,2,73,19491,1360,0.777154,-1.972255,0.937292,-0.591747,1.334833
3,2018-01-13,1,2,73,19490,1360,0.777154,-0.087070,0.937292,-0.449731,1.334833
4,2018-01-13,1,2,73,19489,1360,0.777154,0.705565,0.937292,-0.165698,1.334833


In [196]:
data.shape

(180516, 11)

In [199]:
data=data.dropna()

In [200]:
data.isnull().sum()

Order_Date                       0
Order Item Quantity              0
Department Id                    0
Product Category Id              0
Order Customer Id                0
Product Card Id                  0
Days for shipment (scheduled)    0
Order Item Profit Ratio          0
Sales                            0
Order Item Discount Rate         0
Order Item Product Price         0
dtype: int64

In [201]:
data.to_csv('Dataset/Cleaned_data.csv',  header=True,index = False)